In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import concurrent.futures

问题：
1. current price 换成 2021-11-26
2. threads
3. weights? treat pos and neg as two portfolios, and use combine portfolio to get the optimal weight
4. monthly return or weekly return?



## Group Assignment
### Team Number: 4
### Team Member Names: Jacky Xu, Esha Kumar, Jingyi Fang
### Team Strategy Chosen: SAFE

In [2]:
# read the ticker file
tickers = pd.read_csv("Tickers.csv",header=None)
# rename the column as "ticker"
tickers.columns=['ticker']
# create a list that contains all the tickers
ticker_lst = tickers['ticker'].tolist()


In [3]:
yf.Ticker("AAPL").history(start = '2021-11-23', end = '2021-11-24',interval='1d')['Close'][0]

161.41000366210938

In [4]:
# call_tickers takes a list of tickers and return a list of called tickers
def call_tickers(lst):
    # create an empty list to store the called tickers
    called_tickers= []
    # iterate the ticker list
    for i in lst:
        # call each ticker and append it to the called ticker list
        called_tickers.append(yf.Ticker(i))
    # return the called ticker list when finish
    return called_tickers

# get_ticker_info takes a called tickers, an empty dataframe, an empty list 
# and store the currency and current price into the dataframe, append the called ticker into the list
def get_ticker_info(i):
    ticker_name = str(i)[24:-1]
    #return ticker_name
    # use try and except to ignorethe delisted stocks
    try:
        # extract currency and current price from the info dictionary and store then into the dataframe
        currency =  i.info['currency']
        avg_daily_volumn = i.history(start = "2021-07-02", end = "2021-10-22" ,interval='1d')['Volume'].mean()
        price1126 = i.history(start = '2021-11-24', end = '2021-11-25',interval='1d')['Close'][0]
        return [ticker_name,currency,avg_daily_volumn,price1126]
        #print(ticker_name)
        # append the ticker into the ticker list
        #lst.append(i)
    except:
        pass


# get_prices takes a list of called tickers, start date adn end date, returns the historical prices in the given time period
def get_prices(lst,start,end):
    # open an empty dataframe
    prices = pd.DataFrame()
    # iterate the ticker list
    for i in lst:
        # extract the historical closing prices and store into the dataframe
        prices[str(i)[24:-1]] = i.history(start = start,end=end,interval = '1d')['Close']
    # return the prices
    return prices

# get_weekly_returns takes the historical prices(or portfolio value) and returns the weekly returns
# usually we use monthly returns but since the competition only lasts for 6 days, we decided to use the weekly returns
def get_weekly_returns(prices):
    weekly_returns = prices.resample('W').first().pct_change()
    weekly_returns = weekly_returns.iloc[1:]
    return weekly_returns

# we still use monthly returns to calculate the beta and others
def get_monthly_returns(prices):
    monthly_returns = prices.resample('MS').first().pct_change()
    #monthly_returns = monthly_returns.iloc[1:]
    return monthly_returns
    
# get_std takes the prices of the stock and returns the std of the prices
def get_std(prices):
    # open an empty dataframe with column names set up
    std = pd.DataFrame(columns = ('ticker','std'))
    # set the index
    index = 0
    # iterate through the ticker names
    for i in prices.columns:
        # store the ticker name and the std of prices
        std.loc[index] = [i, prices[i].std()]
        # add 1 to the index
        index += 1
    # return the std
    return std


# rank takes a dataframe, column name and method and rank the dataframe by the given column using the given method
def get_rank(df,column_name,method):
    # use the build-in rank function to rank the dataframe, store the rankings in a new column
    df[column_name+'_rank'] = df[column_name].rank(method=method)
    # return the ranked dataframe
    return df

# get_std_beta_rank takes the ranked std dataframe and the ranked beta dataframe, rank the dataframe by both and return the top 40 stocks
def get_std_beta_rank(std,beta):
    # combine the std dataframe and the beta dataframe
    rank = pd.concat([std,beta['beta'],beta['beta_rank']],join='inner',axis=1)
    # add up the ranks to get the final rank
    rank['final_rank'] = rank['std_rank']*8 + rank['beta_rank']
    # rank by the final rank
    rank = rank.sort_values(by='final_rank')
    # takes the top 40 stocks
    rank = rank[:40]
    # return the dataframe
    return rank



In [5]:
# set up the start date and the end date
start_date = "2021-07-02"
end_date = "2021-10-22"

# call the tickers
called_tickers = call_tickers(ticker_lst)

In [6]:
# open an empty dataframe and a list to store the ticker info and the tickers
ticker_info = pd.DataFrame()
tickers = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    result = executor.map(get_ticker_info, called_tickers)
for i in result:
    try:
        if i[1] == 'USD':
            if i[2] > 10000:
                ticker_info[i[0]] = [i[3]]
    except:
        pass
ticker_info = ticker_info.T
ticker_info.columns = ['11-26_price']

In [7]:
len(ticker_info)

55

In [8]:
good_tickers = call_tickers(ticker_info.index)
# extract the closing prices for the good tickers
prices = get_prices(good_tickers, start_date, end_date)

# Get a list of Ticker Symbols (list of Strings)
ticker_symbols = prices.columns

# calculate the std according to the closing prices
std = get_std(prices)

# rank them in ascending order
std = get_rank(std,'std','min')

In [9]:
len(good_tickers)

55

In [10]:
# calculate the std according to the closing prices
std = get_std(prices)

# rank them in ascending order
std = get_rank(std,'std','min')

In [11]:
# Calculating Beta
def get_beta(good_tickers, prices, start_date, end_date, ticker_symbols):
    Ticker = '^GSPC'
    MarketIndex = yf.Ticker(Ticker) # The symbol yfinance uses for the S&P 500

    MarketIndex_hist = MarketIndex.history(start=start_date, end=end_date)

    # DataFrame for Market Index
    marketDF = pd.DataFrame(MarketIndex_hist['Close'])
    marketDF.columns = [Ticker]
    

    # Loop iterates through the column of prices
    betaList = []
    for i in range(len(ticker_symbols)):
        currentStockPrice = prices[ticker_symbols[i]]
        currentStockPrice = pd.concat([currentStockPrice, marketDF], join = 'inner', axis=1)
        
        # Getting the Monthly Return of each stock
        monthly_returns = currentStockPrice.resample('MS').first().pct_change()  # Dropping the first entry (since it's N/A)
        monthly_returns.drop(index=monthly_returns.index[0], inplace=True)
        
        
        # Calculate the market variance (you will need to reference the column correponding to the market)
        MarketVar = monthly_returns[Ticker].var()
        
        betaList.append(monthly_returns.cov() / MarketVar)
        
    # Filter through betaList and extract the beta for each stock
    for i in range(len(betaList)):
        betaList[i] = betaList[i].iat[0,1]
    
    # Creating a DataFrame for Tickers and their Beta value
    beta = pd.DataFrame(columns = ('ticker','beta'))
    for i in range(len(good_tickers)):
        beta.loc[i] = [str(good_tickers[i])[24:-1], betaList[i]]

    return beta

In [12]:
# get_correlation takes a ticker(self) and return 
def get_correlation(self,std_rank,prices,length):
    rank = pd.DataFrame()
    self_weekly_returns = pd.DataFrame(get_weekly_returns(prices[self]))
    for i in range(1,length):
        candidate = std_rank.index[i]
        candidate_weekly_returns = pd.DataFrame(get_weekly_returns(prices[candidate]))
        corr = self_weekly_returns[self].corr(candidate_weekly_returns[candidate])  
        rank[candidate] = [corr]
    return rank.T

# pair_by_corr takes the ranked dataframe and the prices of the stocks, 
# returns 10 portfoliois in one dataframe, each with a pair of stocks with the lowest correlation
def pair_by_corr(rank, prices):
    # open an empty dataframe
    pairs = pd.DataFrame()
    # counter is the number of pairs
    counter = 0
    # loop until we have 10 pairs
    while counter < 10:
        # self is the stock that has lowest std and beta(which is the safest one)
        self = rank.index[0]
        # get the weekly_returns for self
        self_weekly_returns = pd.DataFrame(get_weekly_returns(prices[self]))
        # for all other stocks in the ranked dataframe
        corrs = get_correlation(self,rank,prices,len(rank)-1)
        corrs.columns = ['corr']
        corrs = corrs.sort_values(by='corr')
        champion = corrs.index[0]
        champion_weekly_returns = pd.DataFrame(get_weekly_returns(prices[champion]))
        # get current price from the ticker info dataframe
        self_price = prices[self][0]
        champion_price = prices[champion][0]
        # calculate the number of shares we could purchase using the current price
        self_shares = 100000 /  self_price
        champion_shares = 100000 /  champion_price
        # get weekly price, portfolio value, and portfolio return for self and champion
        pairs[self+'_price'] = prices[self].resample('W').first()
        pairs[self+'_value'] =  pairs[self+'_price']*self_shares
        pairs[self+'_return'] = pd.DataFrame(get_weekly_returns(pairs[self+'_value']))
        pairs[champion+'_price'] = prices[champion].resample('W').first()
        pairs[champion+'_value'] =  pairs[champion+'_price']*champion_shares
        pairs[champion+'_return'] = pd.DataFrame(get_weekly_returns(pairs[champion+'_value']))
        # remove self and champion from the ranked dataframe to prepare for the next iteration
        rank = rank[1:]
        rank = rank[rank.index != str(champion)]
        # add 1 to the counter for each finished pair
        counter += 1
    # return the pairs
    return pairs


def get_difference(df): 
    return max(abs(df['value'].max()-100000) , abs(df['value'].min()-100000))

# when 'stock' is True, create_portfolio takes two stocks and the pairs dataframe, 
# returns a weight and the weighted portfolio with the lowest sharpe ratio
# when 'stock' is False, create_portfolio takes two portfolio(old and new), the min and max range for the sharpe ratio,
# returns a weight and the weighted portfolio with the lowest sharpe ratio
def create_portfolio(old,new,range_min,range_max,stock,stock1,stock2,pairs):
    # open an empty portfolio 
    candidate = pd.DataFrame()
    champion = 0
    # min_ratio stores the lowest sharpe ratio
    min_difference = ['+' , 100000]
    # weight_1 is the weight of stock1 or the portfolio weight for old
    weight_1 = 0
    # iterates the given range
    for i in range(range_min,range_max+1):
        if stock:
            # for the stock version, the porfolio value is taken from the pairs dataframe
            # multiply the weight i with the value of each stock
            # add them up and double the result(since each stock value starts from 500000 and the required amount is 1 million)
            candidate['value'] = pairs[stock1+'_value']*(i/100)+pairs[stock2+'_value']* (1-i/100)
        else:
            # for the portfolio version, just multiply the portfolio values by the weights and add them up
            candidate['value'] = old['value']*(i/100)+new['value']* (1-i/100)
        # get the monthly returns 
        # get the sharpe ratio 
        difference = get_difference(candidate)
        #print(difference)
        # if the absolute value of the sharpe ratio(since the ratio could be negative too) is less than the min_ratio
        if abs(difference) <= min_difference[1]:
            if candidate['value'].median() > 100000:
                 min_difference = ['+',abs(difference)]
            # replace the min_ratio by the the absolute value of the current sharpe ratio 
            else:
                min_difference = ['-',abs(difference)]
            # update the champion and the weight
            champion = candidate
            weight_1 = i
    # return the optimal weight and the portfolio with the optimal weight
    return weight_1, min_difference, champion

# get_range takes the optimal weight for stock1 and returns the range for the portfolio weight
def get_range(stock1_weight):
    if (stock1_weight < 1 or stock1_weight > 99):
        range_max = 1
        range_min = 0
    # if stock1 has a greater weight, the max portfolio weight would be 35/stock1_weight
    # since 35 is the max weight, and the weight of each stock is given by the product of stock1_weight * portfolio weight
    # so the max portfolio weight is 35/stock1_weight. Same logic for the min portfolio weight
    else:
        if stock1_weight >= 50:
            range_max = 35/stock1_weight
            range_min = 2.5/(100-stock1_weight)
            # if stock1 has a lower weight, just flip the stock1 and the other stock
        else:
            range_max = 35/(100-stock1_weight) 
            range_min = 2.5/stock1_weight
    
    if range_min < range_max:
        return math.ceil(range_min*100),math.floor(range_max*100)
    else:
        return math.floor(range_max*100),math.ceil(range_min*100)


        

In [13]:
# calculate the beta according to the closing prices
beta = get_beta(good_tickers, prices, start_date, end_date, ticker_symbols)

# rank them in ascending order
beta = get_rank(beta,'beta','min')

# combine the rank of std and beta, then rank the combined rank
std_beta_rank = get_std_beta_rank(std,beta)
std_beta_rank.set_index('ticker',inplace=True)
std_beta_rank.head()

,std,std_rank,beta,beta_rank,final_rank
ticker,,,,,
MON,0.046874,1.0,-0.146849,13.0,21.0
T,0.663311,2.0,-0.460408,11.0,27.0
KMI,0.760351,3.0,-1.738541,5.0,29.0
KO,1.205896,5.0,1.013712,28.0,68.0
MO,1.189154,4.0,1.697517,42.0,74.0


In [74]:
# get the portfolio of pairs 
pairs = pair_by_corr(std_beta_rank,prices)
pairs.head()

,MON_price,MON_value,MON_return,PFE_price,PFE_value,PFE_return,T_price,T_value,T_return,PEP_price,...,USB_return,SO_price,SO_value,SO_return,C_price,C_value,C_return,CL_price,CL_value,CL_return
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-04,9.655,100000.000000,NaN,39.030785,100000.000000,NaN,28.158052,100000.000000,NaN,147.896271,...,NaN,60.274460,100000.000000,NaN,69.385284,100000.000000,NaN,80.773155,100000.000000,NaN
2021-07-11,9.730,100776.797632,0.007768,38.598526,98892.518789,-0.011075,27.955755,99281.565332,-0.007184,148.114777,...,-0.027098,60.382214,100178.771828,0.001788,67.217300,96875.441202,-0.031246,81.040092,100330.477688,0.003305
2021-07-18,9.700,100466.080555,-0.003083,39.060253,100075.500753,0.011962,27.937721,99217.519775,-0.000645,148.492172,...,0.028031,60.538948,100438.806455,0.002596,68.429405,98622.360318,0.018033,81.989204,101505.511566,0.011712
2021-07-25,9.670,100155.363477,-0.003093,39.443394,101057.137602,0.009809,27.260860,96813.728495,-0.024227,154.739365,...,-0.053317,61.508747,102047.778222,0.016019,64.132851,92430.046485,-0.062788,83.017410,102778.466509,0.012541
2021-08-01,9.675,100207.151303,0.000517,41.074177,105235.334626,0.041345,27.643435,98172.395193,0.014034,156.000717,...,0.030637,62.429565,103575.487191,0.014971,66.783699,96250.522845,0.041334,83.583977,103479.895673,0.006825


In [149]:
bigNum = []
for i in range (len(pairs)):
    bigNum.append(1230918230)
lowest = pd.DataFrame({"final": bigNum})
for w in range(49):
    # open empty lists for the protfolio weights, differences and symbols
    portfolio_weights = []
    diffs = []
    stock1_symbols = []
    stock2_symbols = []
    # extract the column names for prices in the pairs dataframe( 'tickername_price')
    symbols = pairs.filter(like='_price', axis=1).columns
    # get number of pairs as the number of for loop
    num_of_pairs = round(len(symbols)/2)

    # then iterates the stocks in paris
    for i in range(num_of_pairs):
        # get the ticker names for the pair of stocks, and append to the corresponding symbol lists
        stock1 = symbols[0][:-6]
        stock1_symbols.append(stock1)
        stock2 = symbols[1][:-6]
        stock2_symbols.append(stock2)

        # combine the pair of stock, store the optimal stock weight, difference, and the portfolio with optimal weight
        w1, diff, champion = create_portfolio(0,0,1+w,99-w,True,stock1,stock2,pairs)
        # append the result to their corresponding lists
        portfolio_weights.append(w1)
        diffs.append(diff)
        # remove the finished pair
        symbols = symbols[2:]

        # format the differences with signs
    for i in range(num_of_pairs):
        if diffs[i][0] == '+':
            diffs[i] = diffs[i][1]
        else:
            diffs[i] = (diffs[i][1])*-1

    # create a dataframe with the weights and the differences
    weights = pd.DataFrame(portfolio_weights, diffs)
    # format the dataframe, calculate the stock 2 weight and add the symbols in
    weights.reset_index(inplace=True)
    weights.columns = ['diff','stock1_weight']
    weights['stock1_symbol'] = stock1_symbols
    weights['stock2_weight'] = 100 - weights['stock1_weight']
    weights['stock2_symbol'] = stock2_symbols
    weights
    # separate the weights dataframe by signs

    pos = weights[weights['diff'] > 0]
    neg = weights[weights['diff'] < 0]
    # calculate the sum of positive differences and negative diffrences
    pos_diff = pos['diff'].sum()
    neg_diff = neg['diff'].sum()
    # calculate the total differences
    total_diff = pos_diff - neg_diff
    # calculate the weights of positve and negative diffserences
    pos_weight = pos_diff/total_diff
    neg_weight = (neg_diff/total_diff)*-1
    #print(pos_weight, neg_weight)
    # each stock must make up a minimum of (100/(2n))% of the portfolio , add/subtract 0.01 just in case
    min_weight = ((100/(len(weights)*4))+0.01)/(1+w)
    max_weight = (35-0.01)/(99-w)
    #print(min_weight,max_weight)
    pos_weight/len(pos)

    # the minimal weight for the entire pos/neg differences is given multiplying the minimal weight of each stock 
    # and the number of stocks that has pos/neg differences, then divided by 30, since 
    # 30 is the minimal weight of how each stock is weighted in its local portfolio
    # use 0.5 since we wish both positive and negative differences make up of 50% of the total differences
    # so that they can cancel each other
    if pos_weight < neg_weight:
        # make sure that the weights is greater than the min weight
        neg_weight = min(min_weight*len(neg),0.5-0.5/pos_weight)
        #pos_weight = min(0.5/pos_weight,min_pos_weight)
        pos_weight = 1 - neg_weight
    else:
        pos_weight = min_weight*len(pos)
        neg_weight = 1 - pos_weight


    pos_weight, neg_weight
    max_weight,min_weight

    # calculate the final weight of each stock
    pos['stock2_weight']*= pos_weight/len(pos)
    pos['stock1_weight']*= pos_weight/len(pos)
    neg['stock2_weight'] *= neg_weight/len(neg)
    neg['stock1_weight'] *= neg_weight/len(neg)

    # combine the pos and neg, as well as stock1 and stock2
    weights = pd.concat([pos,neg],join='inner',axis=0)
    weights_1 = weights[['stock1_weight','stock1_symbol']]
    weights_2 = weights[['stock2_weight','stock2_symbol']]
    weights_1.columns = ['weight','symbol']
    weights_2.columns = ['weight','symbol']
    weights = pd.concat([weights_1,weights_2],join = 'inner',axis = 0)
    weights.reset_index(inplace = True)
    weights.drop('index',axis = 1)

    # initialize the final portfolio
    final_portfolio = pd.DataFrame(columns = ['Ticker', 'Price', 'Shares', 'Value', 'Weight'])
    # fill in the required information
    for i in range(len(weights)):
        ticker = weights['symbol'][i]
        price = pairs[ticker+"_price"][0]
        weight = weights['weight'][i]
        value = weight/100*100000
        shares = value/price
        final_portfolio.loc[i+1] = [ticker,price,shares,value,weight]

    ### Check the final portfolio, using the price of start date to buy shares
    ticker = final_portfolio['Ticker']
    ticker_lst = ticker.tolist()
    shares = final_portfolio['Shares']
    price = pairs[ticker+'_price']
    for i in range(len(ticker_lst)):
        ticker_name = ticker_lst[i]
        price[ticker_name+'_price'] *= final_portfolio.loc[i+1,'Shares']

    price['final'] = price.sum(axis = 1)
    price.tail()
    if price['final'].iloc[len(price)-1] < lowest['final'].iloc[len(price)-1]:
        lowest = price.copy()

<ipython-input-149-6be532b52d8a>:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos['stock2_weight']*= pos_weight/len(pos)
<ipython-input-149-6be532b52d8a>:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos['stock1_weight']*= pos_weight/len(pos)
<ipython-input-149-6be532b52d8a>:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [150]:
pos

,diff,stock1_weight,stock1_symbol,stock2_weight,stock2_symbol
0,13058.421465,2.612449,MON,2.510000,PFE
2,5173.015366,2.612449,KMI,2.510000,NEE
3,3485.179895,2.612449,KO,2.510000,MRK
5,5961.340797,2.612449,CVS,2.510000,PG
8,4890.634749,2.510000,USB,2.612449,SO


In [153]:
neg

,diff,stock1_weight,stock1_symbol,stock2_weight,stock2_symbol
1,-2111.046192,7.587551,T,7.290000,PEP
4,-5197.777411,7.587551,MO,7.290000,ABBV
6,-8655.922277,7.290000,SLB,7.587551,ABT
7,-5731.465417,7.587551,CSCO,7.290000,GM
9,-4959.167479,7.587551,C,7.290000,CL


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.

In [155]:
lowest

,MON_price,KMI_price,KO_price,CVS_price,USB_price,T_price,MO_price,SLB_price,CSCO_price,C_price,...,NEE_price,MRK_price,PG_price,SO_price,PEP_price,ABBV_price,ABT_price,GM_price,CL_price,final
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-04,2612.448980,2612.448980,2612.448980,2612.448980,2510.000000,7587.551020,7587.551020,7290.000000,7587.551020,7587.551020,...,2510.000000,2510.000000,2510.000000,2612.448980,7290.000000,7290.000000,7587.551020,7290.000000,7290.000000,100000.000000
2021-07-11,2632.742421,2594.160625,2597.983613,2572.832784,2441.985274,7533.039423,7450.262708,6929.835049,7508.189577,7350.473527,...,2538.722473,2494.352372,2511.477554,2617.119302,7300.770439,7325.446959,7637.688682,7104.535275,7314.091823,98937.912104
2021-07-18,2624.625096,2605.414976,2626.914346,2593.116409,2510.436078,7528.179934,7581.165734,6940.951443,7543.618803,7483.021907,...,2537.370827,2476.469509,2532.902081,2623.912574,7319.372761,7445.712190,7637.045752,7291.236696,7399.751793,99813.113977
2021-07-25,2616.507771,2416.902180,2687.186706,2556.669255,2376.586680,7345.791044,7469.419808,5924.931164,7519.527082,7013.176935,...,2601.573743,2460.502807,2593.851594,2665.946141,7627.305018,7387.271808,7664.830604,6698.985895,7492.550209,97656.050597
2021-08-01,2617.860702,2508.344783,2751.316793,2642.300163,2449.397256,7448.880573,7600.322214,6549.661099,7861.066208,7303.057528,...,2598.870711,2466.569874,2607.191238,2705.856758,7689.478729,7538.960000,7711.933003,6895.578522,7543.684395,100131.737449
2021-08-08,2625.978027,2473.913394,2742.637462,2627.633072,2403.618307,7430.377509,7619.478694,6336.230506,7858.231696,7353.247588,...,2663.073626,2432.720063,2636.928359,2720.717072,7652.762214,7389.191880,7802.266881,7051.368683,7135.505364,99758.270515
2021-08-15,2632.742421,2433.919726,2731.547422,2603.400378,2486.892570,7361.651328,7579.568947,6222.845330,7861.066208,7764.716309,...,2722.208130,2405.257130,2642.503855,2753.409993,7556.319404,7300.227042,7946.801579,6670.548027,7027.853721,99634.641842
2021-08-22,2621.919235,2428.206068,2771.568195,2722.649745,2510.000000,7454.167677,7724.838713,6176.157242,7975.856877,7827.685403,...,2836.759851,2488.604379,2688.224693,2846.961781,7737.455318,7578.641991,8043.587982,6546.904877,7157.035692,101255.171674
2021-08-29,2608.390446,2325.364378,2721.421443,2697.460859,2448.961178,7274.421568,7739.206072,6111.683519,8296.138250,7696.319233,...,2851.965606,2502.974484,2682.834961,2859.399552,7727.664147,7639.445273,8119.726006,5957.126996,7031.441879,100474.915360
